## # Introduction
<p><img src="https://assets.datacamp.com/production/project_981/img/lego_unsplash.jpg" alt="A picture of Lego characters!"></p>
<h3 id="letslookatlegosets">Let's look at Lego sets!</h3>
<p>Lego is a household name across the world, supported by a diverse toy line, hit movies, and a series of successful video games. In this project, we are going to explore a key development in the history of Lego: the introduction of licensed sets such as Star Wars, Super Heroes, and Harry Potter.</p>
<p>It may not be widely known, but Lego has had its share of ups and downs since its inception in the early 20th century. This includes a particularly rough period in the late 90s. As described in <a href="https://www.businessinsider.com/how-lego-made-a-huge-turnaround-2014-2?r=US&IR=T">this article</a>, Lego was only able to survive due to a successful internal brand (Bionicle) and the introduction of its first licensed series: Star Wars. In the instructions panel are the two questions you will need to answer to complete this project.</p>
<p>Before diving into our analysis though, let's become familiar with the two datasets that will help you with this project:<br><br></p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/lego_sets.csv</b></div>
<ul>
    <li><b>set_num:</b> A code that is unique to each set in the dataset. <b><i>This column is critical, and a missing value indicates the set is a duplicate or invalid!</i></b></li>
    <li><b>set_name:</b> A name for every set in the dataset (note that this can be the same for different sets).</li>
    <li><b>year:</b> The date the set was released.</li>
    <li><b>num_parts:</b> The number of parts contained in the set.<b><i> This column is not central to our analyses, so missing values are acceptable.</i></b></li>
        <li><b>theme_name:</b> The name of the sub-theme of the set.</li>
    <li><b>parent_theme:</b> The name of the parent theme the set belongs to. Matches the `name` column of the `parent_themes` csv file.</li>
</ul>

<div style="font-size:20px"><b>datasets/parent_themes.csv</b></div>
<ul>
    <li><b>id:</b> A code that is unique to every theme.</li>
    <li><b>name:</b> The name of the parent theme.</li>
    <li><b>is_licensed:</b> A Boolean column specifying whether the theme is a licensed theme.</li>
</ul>
    </div>
<p>From here on out, it will be your task to explore and manipulate the existing data until you are able to answer the two questions described in the instructions panel. Feel free to add as many cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><em><strong>Note:</strong> If you haven't completed a DataCamp project before you should check out the <a href="https://projects.datacamp.com/projects/33">Intro to Projects</a> first to learn about the interface. In this project, you also need to know your way around <code>pandas</code> DataFrames and it's recommended that you take a look at the course <a href="https://www.datacamp.com/courses/data-manipulation-with-pandas">Data Manipulation with pandas</a>.</em></p>

In [1]:
# Use this cell to begin your analyses, and add as many cells as you would like!
# Importing libraries that might be useful
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading both datasets into dataframes
lego_sets = pd.read_csv("datasets/lego_sets.csv")
parent_themes = pd.read_csv("datasets/parent_themes.csv")

## Part 1: What percentage of all licensed sets ever released were Star Wars themed?

We can begin this section by peeking at the top few rows from both of our dataframes, to get an idea of what kind of data we are dealing with

In [3]:
lego_sets.head()

,set_num,name,year,num_parts,theme_name,parent_theme
0,00-1,Weetabix Castle,1970,471.0,Castle,Legoland
1,0011-2,Town Mini-Figures,1978,NaN,Supplemental,Town
2,0011-3,Castle 2 for 1 Bonus Offer,1987,NaN,Lion Knights,Castle
3,0012-1,Space Mini-Figures,1979,12.0,Supplemental,Space
4,0013-1,Space Mini-Figures,1979,12.0,Supplemental,Space


In [4]:
parent_themes.head()

,id,name,is_licensed
0,1,Technic,False
1,22,Creator,False
2,50,Town,False
3,112,Racers,False
4,126,Space,False


We can follow by inspecting the total missing values for each of our dataframe's columns. Since the `set_num` column is critical for our analysis, and a missing value in this row represents a duplicate or invalid set, we need to make sure none of the entries have a missing value in this column.

In [5]:
# Checking how many missing values are there per columns
lego_sets.isna().sum()

set_num          153
name             153
year               0
num_parts       5060
theme_name       153
parent_theme       0
dtype: int64

Looks like the `set_num`, `name` and `theme_name` columns have the same amount of missing entries. It might be possible that the entries are all the same, let's if this is the case.

In [6]:
# Let's check some of the missing rows/entries
lego_sets[lego_sets["set_num"].isna()].sample(10)

,set_num,name,year,num_parts,theme_name,parent_theme
11842,NaN,NaN,2013,0.0,NaN,Super Heroes
11919,NaN,NaN,2002,NaN,NaN,Harry Potter
11904,NaN,NaN,2005,NaN,NaN,Harry Potter
11966,NaN,NaN,2015,NaN,NaN,Super Heroes
11909,NaN,NaN,2014,55.0,NaN,Disney Princess
11982,NaN,NaN,2006,NaN,NaN,Super Heroes
11933,NaN,NaN,2017,0.0,NaN,Super Heroes
11859,NaN,NaN,2001,298.0,NaN,Harry Potter
11899,NaN,NaN,2017,NaN,NaN,Super Heroes
11981,NaN,NaN,2014,NaN,NaN,Minecraft


Looks like these columns have all the same missing entries, so we can go ahead and drop the rows with missing `set_num` values, which should also clean all the missing values from the other 2 columns.

In [7]:
lego_sets.dropna(subset=["set_num"],inplace=True)

In [8]:
lego_sets.isna().sum()

set_num            0
name               0
year               0
num_parts       4998
theme_name         0
parent_theme       0
dtype: int64

Looks like there are no more missing `set_num` values, and the `name` and `theme_name` columns were also cleared of missing values too. The `num_parts` columns still has many missing values, but since the column is not critical to our analysis, we can leave it in this state.

Moving on, we can get the total number of Star Wars sets ever released by subsetting our dataframe to rows in which the `parent_theme` is `Star Wars`, and then counting the number of rows, since we already deleted any duplicate entries that might have been present.

In [9]:
star_wars_sets = lego_sets[lego_sets["parent_theme"] == "Star Wars"].shape[0]

Now, in order to get the percentage of the Star Wars sets in relation to the total number of licensed sets ever released, we need to count all of the sets in which the `parent_theme` a licensed theme.

One way to do this would be to generate a list of licensed parent themes from the `parent_themes` dataframe and subseting our `lego_set` dataframe on the `parent_themes` column:

In [10]:
licensed_parent_themes = list(parent_themes[parent_themes["is_licensed"] == True]["name"].unique())

In [11]:
licensed_lego_sets = lego_sets[lego_sets["parent_theme"].isin(licensed_parent_themes)]
licensed_lego_sets.head()

,set_num,name,year,num_parts,theme_name,parent_theme
44,10018-1,Darth Maul,2001,1868.0,Star Wars,Star Wars
45,10019-1,Rebel Blockade Runner - UCS,2001,NaN,Star Wars Episode 4/5/6,Star Wars
54,10026-1,Naboo Starfighter - UCS,2002,NaN,Star Wars Episode 1,Star Wars
57,10030-1,Imperial Star Destroyer - UCS,2002,3115.0,Star Wars Episode 4/5/6,Star Wars
95,10075-1,Spider-Man Action Pack,2002,25.0,Spider-Man,Super Heroes


We can now easily get the total number of licensed sets released:

In [12]:
total_licensed_sets = licensed_lego_sets.shape[0]
total_licensed_sets

1179

In [13]:
the_force = int(100 * star_wars_sets / total_licensed_sets)
the_force

51

## Part 2: In which year was Star Wars not the most popular licensed theme (in terms of number of sets released that year)?

In [14]:
grouped_lego_sets = licensed_lego_sets.groupby("year")

In [15]:
top_sets = grouped_lego_sets["parent_theme"].agg(pd.Series.mode)
top_sets

year
1999       Star Wars
2000       Star Wars
2001       Star Wars
2002       Star Wars
2003       Star Wars
2004       Star Wars
2005       Star Wars
2006       Star Wars
2007       Star Wars
2008       Star Wars
2009       Star Wars
2010       Star Wars
2011       Star Wars
2012       Star Wars
2013       Star Wars
2014       Star Wars
2015       Star Wars
2016       Star Wars
2017    Super Heroes
Name: parent_theme, dtype: object

In [16]:
new_era = top_sets[top_sets != "Star Wars"].index[0]
new_era

2017

## Conclusions

From the previous analysis, we can conclude the following:
1. Over half (51%) of all the licensed lego sets ever released are Star Wars themed sets.
2. Star Wars themed sets were the most popular lego set from 1999 up until 2017, when the most popular lego sets were the Super Heroes sets.